<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Pain_Undo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# --- FINAL RELEASE (Colab Free Tier Attempt): Paints-UNDO Setup and Launch ---
# Adapted from working Kaggle script.
# WARNING: HIGH LIKELIHOOD OF FAILURE due to Colab Free Tier System RAM limitations (~12.7 GB).
# Expect potential crash during model loading after installations complete.

import os
import re
import fileinput
import site # To find site-packages
import sys # To check python version for path fallback

# 1. Check Resources (GPU and RAM)
print("--- Step 0: Checking Resources ---")
!nvidia-smi
print("\nSystem RAM:")
!free -h
print("\n" + "="*30 + "\n")

# Define working directory for Colab
WORKING_DIR = "/content/"
REPO_DIR = os.path.join(WORKING_DIR, "Paints-UNDO")

# 2. Clone the Repository
print("--- Step 1: Cloning Repository ---")
# Remove previous clone if it exists to start fresh
!rm -rf {REPO_DIR}
os.chdir(WORKING_DIR) # Ensure we are in /content before cloning
!git clone https://github.com/lllyasviel/Paints-UNDO.git
%cd {REPO_DIR}
print(f"Changed directory to: {REPO_DIR}")
print("\n" + "="*30 + "\n")

# 3. Install Dependencies (Strategy: Let peft choose transformers, use diffusers==0.29.0, upgrade Gradio, force numpy<2.0)
print("--- Step 2: Installing Dependencies ---")
print("Creating temporary requirements file excluding diffusers, gradio, transformers...")
# Create a temporary requirements file excluding conflicting/managed packages
!grep -v '^diffusers==' requirements.txt | grep -v '^gradio==' | grep -v '^transformers==' > temp_requirements.txt

print("Installing base requirements (including peft, which will pull its transformers)...")
# Use --use-deprecated=legacy-resolver if dependency errors occur, but try without first
!pip install --no-cache-dir -r temp_requirements.txt # Removed legacy-resolver for now

print("\nInstalling specific diffusers version (0.29.0)...")
!pip install --no-cache-dir diffusers==0.29.0

print("\nUpgrading Gradio and Client...")
!pip install --no-cache-dir --upgrade gradio gradio-client

print("\nForce installing NumPy < 2.0 (numpy==1.26.4) for compatibility...")
!pip install --no-cache-dir "numpy<2.0" numpy==1.26.4 --force-reinstall

# Clean up temporary file
!rm temp_requirements.txt
print("--- Dependencies Installation Complete ---")
print("\n--- Verifying Key Package Versions ---")
!pip show gradio transformers huggingface-hub diffusers peft numpy | grep -E '^Name:|^Version:'
print("\n" + "="*30 + "\n")


# 4. Patch the installed transformers library for CLIP Vision model compatibility
print("--- Step 3: Patching installed transformers library ---")
try:
    # Find site-packages directory more reliably
    site_packages_paths = site.getsitepackages()
    transformers_clip_path = None
    py_version_short = f"python{sys.version_info.major}.{sys.version_info.minor}" # e.g., python3.10

    for path in site_packages_paths:
         # Look for the specific file within transformers install location
         potential_path = os.path.join(path, "transformers/models/clip/modeling_clip.py")
         if os.path.exists(potential_path):
              transformers_clip_path = potential_path
              print(f"Found transformers file at: {transformers_clip_path}")
              break

    # Fallback if not found in standard site-packages (adjust python version)
    if not transformers_clip_path:
         site_packages_path = f'/usr/local/lib/{py_version_short}/dist-packages'
         transformers_clip_path = os.path.join(site_packages_path, "transformers/models/clip/modeling_clip.py")
         if os.path.exists(transformers_clip_path):
             print(f"Found transformers file at fallback path: {transformers_clip_path}")
         else:
             transformers_clip_path = None # Explicitly None if not found

    if transformers_clip_path and os.path.exists(transformers_clip_path):
        patch_applied = False
        modified_lines = []
        target_line_found = False

        # Read all lines first
        with open(transformers_clip_path, 'r') as f:
            lines = f.readlines()

        # Find and modify the specific line, preserving indentation
        for i, line in enumerate(lines):
            # Define the argument string to remove more carefully
            arg_to_remove_pattern = r',?\s*interpolate_pos_encoding\s*=\s*interpolate_pos_encoding'
            # Check if the line contains the specific call and the argument
            # Be slightly more robust to variations in the call
            if 'self.embeddings(' in line and 'interpolate_pos_encoding=' in line:
                target_line_found = True
                # Remove the argument using regex substitution
                original_line_strip = line.strip()
                modified_content = re.sub(arg_to_remove_pattern, '', line)
                modified_lines.append(modified_content) # Append the modified line
                print(f"  Original Line {i+1}: {original_line_strip}")
                print(f"  Patched Line {i+1}:  {modified_content.strip()}")
                patch_applied = True
            else:
                modified_lines.append(line) # Keep original line

        if patch_applied:
            # Write the modified content back
            print("Writing patched file...")
            backup_file = transformers_clip_path + '.bak'
            if os.path.exists(backup_file): # Avoid multiple backups piling up
                 os.remove(backup_file)
            os.rename(transformers_clip_path, backup_file) # Create backup
            with open(transformers_clip_path, 'w') as f:
                f.writelines(modified_lines)
            print("--- Successfully patched modeling_clip.py ---")
        elif target_line_found:
             print("--- WARNING: Target line found but patch regex might not have matched. Check manually if issues persist. ---")
        else:
            print("--- WARNING: Target line for patching 'interpolate_pos_encoding' not found in modeling_clip.py. This might be OK if the installed transformers version doesn't use it, or patch needed elsewhere. ---")
    else:
        print(f"--- ERROR: Could not find modeling_clip.py in site-packages to patch. Searched paths: {site_packages_paths} and fallback '/usr/local/lib/{py_version_short}/dist-packages'. ---")

except Exception as e:
    print(f"--- ERROR during patching: {e} ---")
print("\n" + "="*30 + "\n")

# 5. Modify the Gradio launch command to enable sharing
print("--- Step 4: Ensuring share=True in Gradio App ---")
gradio_app_path = 'gradio_app.py'
try:
    with open(gradio_app_path, 'r') as f:
        content = f.read()
    # Use regex for a more robust replacement/addition
    if '.launch(' in content:
        if 'share=True' in content:
            print("share=True already present.")
        else:
            # Try to add share=True inside existing parentheses
            new_content = re.sub(r"(\.launch\([^)]*)(\))", r"\1, share=True\2", content, count=1)
            if new_content != content:
                 with open(gradio_app_path, 'w') as f:
                      f.write(new_content)
                 print("Added share=True to launch() call.")
            else:
                 print("Could not automatically add share=True (pattern mismatch?).")
    else:
         print("WARNING: '.launch(' call not found in gradio_app.py")

    # Verify the change (optional)
    print("--- Checking modification (last few lines of gradio_app.py) ---")
    !tail -n 5 gradio_app.py
    print("\n")

except FileNotFoundError:
    print(f"ERROR: {gradio_app_path} not found.")
except Exception as e:
    print(f"Error modifying {gradio_app_path}: {e}")
print("\n" + "="*30 + "\n")


# 6. Run the Gradio Application
print("--- Step 5: Launching Gradio App on Colab Free Tier ---")
print("Watch System RAM usage closely in 'Runtime -> View resources'.")
print("EXPECTED TO FAIL due to RAM limits during model loading.")
!python gradio_app.py --server_name 0.0.0.0 --enable-queue # Using 0.0.0.0 for consistency, Colab/Gradio will handle tunneling

--- Step 0: Checking Resources ---
Sun Apr  6 16:10:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   41C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+------------


Installing specific diffusers version (0.29.0)...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.3 MB/s eta 0:00:00
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.32.2
    Uninstalling diffusers-0.32.2:
      Successfully uninstalled diffusers-0.32.2

Upgrading Gradio and Client...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 248.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 374.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 314.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 282.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 272.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 217.9 MB/s eta 0:00:00

Force installing NumPy < 2.0 (numpy==1.26.4) for compatibility...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
%cd Paints-UNDO

In [2]:
!python gradio_app.py

/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/usr/local/lib/python3.11/dist-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
2025-04-06 12:37:05.951157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743943026.230163    1972